# 02. Data Munging

## 정규식을 이용한 로그 파싱
* 로그는 다양한 형식으로 저장 (개별 대응이 필요)
* 정규식(re 모듈): 문자열 검색, 대치, 파싱
* StringIO: 문자열 버퍼를 파일 입출력으로 처리

In [1]:
from StringIO import StringIO
import re

log_data = """
221.62.180.95 - - [07/Mar/2014:00:00:06 +0900] "GET /intro/mts/Check.jsp HTTP/1.1" 200 3293
211.24.133.169 - - [07/Mar/2014:00:00:19 +0900] "GET /intro/mts/Check.jsp HTTP/1.1" 200 3293
192.52.91.39 - - [07/Mar/2014:00:00:26 +0900] "GET / HTTP/1.1" 200 1964
"""

regex = '([(\d\.)]+) - - \[([^\[\]:]+:\d+:\d+:\d+) [+-](\d+)\] "(.*?) (.*?) (.*?)" (\d+) (\d+)'

In [2]:
f = StringIO(log_data)

for line in f:
    m = re.match(regex, line)
    if not m is None:
        print m.groups()

('221.62.180.95', '07/Mar/2014:00:00:06', '0900', 'GET', '/intro/mts/Check.jsp', 'HTTP/1.1', '200', '3293')
('211.24.133.169', '07/Mar/2014:00:00:19', '0900', 'GET', '/intro/mts/Check.jsp', 'HTTP/1.1', '200', '3293')
('192.52.91.39', '07/Mar/2014:00:00:26', '0900', 'GET', '/', 'HTTP/1.1', '200', '1964')


## log를 읽어 csv로 저장
* csv.writer(f)
* writer.writerow(row)

In [3]:
import csv
import re

regex = re.compile('([(\d\.)]+) - - \[([^\[\]:]+:\d+:\d+:\d+) [+-](\d+)\] "(.*?) (.*?) (.*?)" (\d+) (\d+)\n')

cols= ['ip', 'time', 'tz', 'method', 'res', 'protocol', 'status', 'len']

log_file = 'data/web_20140408.log'
csv_file = 'data/web_20140408.csv'

In [4]:
f_log = open(log_file, 'rb')
f_csv = open(csv_file, 'wb')

writer = csv.writer(f_csv)
writer.writerow(cols) # write header

for line in f_log:
    m = re.match(regex, line)
    if not m is None:
        writer.writerow(m.groups())
      
f_log.close()
f_csv.close()

## DataFrame 읽기
* pd.read_csv() : csv를 읽어 DataFrame 생성

In [5]:
import numpy as np
import pandas as pd
from pandas import DataFrame

df = pd.read_csv(csv_file, dtype={'status': numpy.int64, 'len': numpy.int64} )

In [6]:
df.columns

Index([u'ip', u'time', u'tz', u'method', u'res', u'protocol', u'status', u'len'], dtype='object')

In [7]:
len(df)

364517

In [8]:
df.head()

,ip,time,tz,method,res,protocol,status,len
0,192.5.90.39,08/Apr/2014:00:00:33,900,GET,/,HTTP/1.1,200,1964
1,211.36.150.107,08/Apr/2014:00:00:44,900,GET,/trapi/mts/Check.jsp,HTTP/1.1,200,3419
2,211.36.145.162,08/Apr/2014:00:00:53,900,GET,/trapi/mts/Check.jsp,HTTP/1.1,200,3419
3,118.34.149.235,08/Apr/2014:00:01:12,900,GET,/common/popup/popup_hts_notice1.html,HTTP/1.1,200,2097
4,118.34.149.235,08/Apr/2014:00:01:13,900,GET,/img/popup/popup_140403_sms.jpg,HTTP/1.1,200,32709


## 컬럼 데이터 포맷 변경
* pandas.to_datetime()
* 문자열을 datetime 타입으로 전환

In [9]:
df['time'] = pd.to_datetime(df['time'], format="%d/%b/%Y:%H:%M:%S")

In [10]:
df.head()

,ip,time,tz,method,res,protocol,status,len
0,192.5.90.39,2014-04-08 00:00:33,900,GET,/,HTTP/1.1,200,1964
1,211.36.150.107,2014-04-08 00:00:44,900,GET,/trapi/mts/Check.jsp,HTTP/1.1,200,3419
2,211.36.145.162,2014-04-08 00:00:53,900,GET,/trapi/mts/Check.jsp,HTTP/1.1,200,3419
3,118.34.149.235,2014-04-08 00:01:12,900,GET,/common/popup/popup_hts_notice1.html,HTTP/1.1,200,2097
4,118.34.149.235,2014-04-08 00:01:13,900,GET,/img/popup/popup_140403_sms.jpg,HTTP/1.1,200,32709


## 컬럼 추출, 인덱스 지정
* pop(): time 컬럼을 추출
* index로 지정

In [11]:
df.index = df.pop('time')

In [12]:
df.head()

,ip,tz,method,res,protocol,status,len
time,,,,,,,
2014-04-08 00:00:33,192.5.90.39,900,GET,/,HTTP/1.1,200,1964
2014-04-08 00:00:44,211.36.150.107,900,GET,/trapi/mts/Check.jsp,HTTP/1.1,200,3419
2014-04-08 00:00:53,211.36.145.162,900,GET,/trapi/mts/Check.jsp,HTTP/1.1,200,3419
2014-04-08 00:01:12,118.34.149.235,900,GET,/common/popup/popup_hts_notice1.html,HTTP/1.1,200,2097
2014-04-08 00:01:13,118.34.149.235,900,GET,/img/popup/popup_140403_sms.jpg,HTTP/1.1,200,32709


## Pickle 포맷으로 저장

In [13]:
df.to_pickle('data/web_20140408.pkl')

In [14]:
df = pd.read_pickle('data/web_20140408.pkl')

In [15]:
df.head()

,ip,tz,method,res,protocol,status,len
time,,,,,,,
2014-04-08 00:00:33,192.5.90.39,900,GET,/,HTTP/1.1,200,1964
2014-04-08 00:00:44,211.36.150.107,900,GET,/trapi/mts/Check.jsp,HTTP/1.1,200,3419
2014-04-08 00:00:53,211.36.145.162,900,GET,/trapi/mts/Check.jsp,HTTP/1.1,200,3419
2014-04-08 00:01:12,118.34.149.235,900,GET,/common/popup/popup_hts_notice1.html,HTTP/1.1,200,2097
2014-04-08 00:01:13,118.34.149.235,900,GET,/img/popup/popup_140403_sms.jpg,HTTP/1.1,200,32709
